In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
from tqdm import tqdm

import nltk
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from functools import lru_cache
from nltk.stem.snowball import SnowballStemmer 

import re
import pymorphy2
from scipy.sparse import csr_matrix
import string
from razdel import tokenize

In [2]:
path = 'data/'

In [3]:
%%time

train = pd.read_parquet(path+'data_fusion_train.parquet')

CPU times: user 21 s, sys: 4.99 s, total: 26 s
Wall time: 19.8 s


Для создания train_unique берем самый популярный класс

In [4]:
train_unique = train[train.category_id != -1]
train_unique = train_unique.groupby(['item_name'])['category_id'].apply(lambda x: pd.Series(x).value_counts().index[0])
train_unique = train_unique.reset_index()

In [5]:
full_unique = train.drop_duplicates('item_name')

Чистим популярные ошибки и раскрываем некоторые сокращения.

In [6]:
dict_1 = {
    'п/об': 'покрытые оболочкой',
    'п/о': 'покрытые оболочкой',
    '%': 'процент',
    'п/сух': 'полусухое',
    'п/сл': 'полусладкое',
}

dict_2 = {
    'г': 'грамм',
    'гр' : 'грамм',
    'л' : 'литр',
    'лит' : 'литр',
    'шт' : 'штука',
    'табл' : 'таблетка',
    'таб' : 'таблетка',
    'порц' : 'порция',
    'охл' : 'охлажденный',
    'уп' : 'упаковка',
    'шок' : 'шоколадный',
    'мол' : 'молочный',
    'жев' : 'жевательный',
    'жеват': 'жевательный',
    'кур' : 'куриная',
    'жен' : 'женские',
    'бут' : 'бутылка',
    'сух' : 'сухой',
    'дет' : 'детский',
    'вар' : 'вареная',
    'паст' : 'пастеризованный',
    'ваф' : 'вафельный',
    'муж' : 'мужской',
    'фас' : 'фасованный',
    'ябл' : 'яблочный',
    'слив' : 'сливочный',
    'газ' : 'газированный',
    'негаз' : 'негазированный',
    'зел' : 'зеленый',
    'фломаст': 'фломастер'
}

In [8]:
def clean_str(s):
    s = s.strip().lower()
    for key in dict_1:
        s = s.replace(key, dict_1[key])
    s = re.sub(" +", " ", s)
    s = re.sub(r'\W+', ' ', s)
    s = re.sub(r'\d+', ' ', s)
    arr = np.array(s.split())
    if len(arr) == 0:
        return ''
    for key in dict_2:
        arr = np.where(arr==key, dict_2[key], arr) 
    s = ' '.join(arr)
    return s

train_unique['item_name_clean'] = train_unique['item_name'].apply(clean_str)
full_unique['item_name_clean'] = full_unique['item_name'].apply(clean_str)

1. Расклеиваем слипшиеся слова
1. Убираем знаки пунктуации
1. Далее приводим к начальной форме и стемматизируем 

In [9]:
stop_rus = set(stopwords.words('russian'))


class Preprocesser_clean:
    def __init__(self):
        lemmatizer = WordNetLemmatizer()
        morph = pymorphy2.MorphAnalyzer()
        stemmer = SnowballStemmer("russian") 
        @lru_cache(maxsize=10**6)
        def lru_morph(word):
            return morph.parse(word)[0].normal_form
        def lru_stemmer(word):
            return stemmer.stem(word)
        def lru_lemmatizer(word):
            return lemmatizer.lemmatize(word)
        self.morph = lru_morph
        self.stemmer = lru_stemmer
        self.lemmatizer = lru_lemmatizer
    def __call__(self, s):
        word_list = tokenize(s)
        word_list = [w.text for w in word_list] 
        word_list = [word for word in word_list if word not in string.punctuation]
        word_list = [self.morph(w) for w in word_list]
        word_list = [self.stemmer(w) for w in word_list]
        word_list = [self.lemmatizer(w) for w in word_list]
        word_list = [word for word in word_list if word not in stop_rus]
        s = ' '.join(word_list)
        s = s.replace(',', '.')
        return s

    
prep_clean = Preprocesser_clean()

train_unique['item_name_clean'] = train_unique['item_name_clean'].apply(prep_clean)
full_unique['item_name_clean'] = full_unique['item_name_clean'].apply(prep_clean)

In [10]:
full_unique = full_unique[['item_name', 'category_id', 'item_name_clean']]

In [11]:
full_unique.shape, train_unique.shape

((3154892, 3), (48225, 3))

In [12]:
train_unique.to_parquet(path+'train_unique.parquet', index=False)
full_unique.to_parquet(path+'full_unique.parquet', index=False)

У нас есть информация по чекам. Мы можем для каждого товара найти топ N самых популярных товаров, которые с ним встречаются. Но train и test не пересекаются по товарам, поэтому можем аналогичную идею сделать для слов.

Создадим словарь: слово - упорядоченный лист близких к нему слов. Что-то типа ассоциативного набора, который будет учитывать близость по информации из чеков.

In [13]:
%%time

train = pd.merge(train, full_unique.drop(columns=['category_id']), on='item_name', how='left')
train = train[['receipt_id', 'item_name_clean']]

CPU times: user 42.5 s, sys: 8.45 s, total: 51 s
Wall time: 53 s


Считаем для всего train вместе с неразмеченными данными. Из-за большого объема приходится считать батчами.

In [14]:
max_recipt_numb = train['receipt_id'].max() // 10 + 1

word_count_total = pd.DataFrame(columns=['word_y', 'count_y'])
receipt_words_total = pd.DataFrame(columns=['word_x', 'word_y', 'count_xy'])

for i in tqdm(range(10)):
    mask = (train['receipt_id'] >= max_recipt_numb * i) & (train['receipt_id'] < max_recipt_numb * (i+1))
    train_batch = train[mask]

    receipt_words = pd.DataFrame(train_batch['item_name_clean'].str.split(' ', 15).tolist())
    receipt_words['receipt_id'] = train_batch['receipt_id']

    del train_batch

    receipt_words = receipt_words.stack().reset_index()
    receipt_words = receipt_words[receipt_words['level_1'] != 'receipt_id']

    del receipt_words['level_1']
    receipt_words.columns = ['receipt_id', 'word']

    receipt_words = receipt_words.drop_duplicates()

    word_count = receipt_words.groupby(['word'])['receipt_id'].count()
    word_count = word_count.reset_index()
    word_count.columns = ['word_y', 'count_y']

    receipt_words = pd.merge(receipt_words, receipt_words, on='receipt_id')

    receipt_words = receipt_words.groupby(['word_x', 'word_y'])['receipt_id'].count()
    receipt_words = receipt_words.reset_index()
    receipt_words = receipt_words.rename(columns={'receipt_id': 'count_xy'})

    receipt_words = receipt_words[receipt_words['word_x'] != receipt_words['word_y']]

    word_count_total = pd.merge(word_count_total, word_count, on=['word_y'], how='outer')
    word_count_total = word_count_total.fillna(0)
    word_count_total['count_y'] = word_count_total['count_y_x'] + word_count_total['count_y_y']
    del word_count_total['count_y_x'] 
    del word_count_total['count_y_y']

    receipt_words_total = pd.merge(receipt_words_total, receipt_words, on=['word_x', 'word_y'], how='outer')
    receipt_words_total = receipt_words_total.fillna(0)
    receipt_words_total['count_xy'] = receipt_words_total['count_xy_x'] + receipt_words_total['count_xy_y']
    del receipt_words_total['count_xy_x'] 
    del receipt_words_total['count_xy_y']
    

100%|██████████| 10/10 [17:27<00:00, 104.74s/it]


In [15]:
receipt_words_total = pd.merge(receipt_words_total, word_count_total, on='word_y', how='left')
receipt_words_total['perc_xy'] =  receipt_words_total['count_xy'] / (receipt_words_total['count_y'])
receipt_words_total = receipt_words_total.sort_values(by=['word_x', 'perc_xy'], ascending=[True, False])

Получили датафрейм со следующими столбцами:
- word_x и word_y - пара слов
- count_xy - количество чеков, в котором встречается word_x и word_y вместе
- count_y - количество чеков, в котором встречается word_y 
- perc_xy - отношение count_xy / count_y

Если мы будем оценивать "близость" слов по count_xy, то везде окажется "пакет", как товар, который в принципе чаще всего встречается в чеках. Поэтому мы нормируем и смотрим perc_xy.

In [16]:
receipt_words_total.sample(10)

,word_x,word_y,count_xy,count_y,perc_xy
5800615,вк,летн,3.0,21694.0,0.000138
849438,барх,челноч,5.0,21.0,0.238095
8319883,x,гидрокситриптофа,1.0,42.0,0.023810
3477440,универс,саморез,688.0,20172.0,0.034107
6359458,скатерт,беж,8.0,2167.0,0.003692
1321542,девя,ст,3.0,274399.0,0.000011
6583315,b,блондирова,1.0,6.0,0.166667
3043705,сверхпрочн,бол,1.0,19912.0,0.000050
1427575,ест,sr,1.0,743.0,0.001346
4694904,кутивейт,процент,3.0,1295439.0,0.000002


In [26]:
receipt_words_total[receipt_words_total['word_x'] == 'пицц'].head(15)

,word_x,word_y,count_xy,count_y,perc_xy
2636508,пицц,a х a йнц грамм,3.0,3.0,1.0
2636520,пицц,bonaza,17.0,17.0,1.0
2636540,пицц,felician,29.0,29.0,1.0
2636557,пицц,kinkku,2.0,2.0,1.0
2636561,пицц,latrattoria,372.0,372.0,1.0
2636585,пицц,pescara,6.0,6.0,1.0
2636594,пицц,roncadin,22.0,22.0,1.0
2636598,пицц,salchicon,22.0,22.0,1.0
2636604,пицц,svila,33.0,33.0,1.0
2636606,пицц,tashir,19.0,19.0,1.0


Чтобы не было примеров, что 5 из 5 раз встретилось что-то странное, немного обрежем данные снизу и сверху.

In [27]:
mask = (receipt_words_total['count_y'] > 100) & (receipt_words_total['perc_xy'] < 0.999)
receipt_words_total_new = receipt_words_total[mask]

In [28]:
receipt_words_total_new[receipt_words_total_new['word_x'] == 'пицц'].head(15)

,word_x,word_y,count_xy,count_y,perc_xy
2637664,пицц,соррентин,383.0,385.0,0.994805
2637562,пицц,ристорант,2471.0,2498.0,0.989191
2636601,пицц,sorrento,250.0,254.0,0.984252
2637987,пицц,эстат,135.0,138.0,0.978261
2637530,пицц,пульчинелл,162.0,168.0,0.964286
2637561,пицц,рист,387.0,419.0,0.923628
2637687,пицц,стаджион,109.0,119.0,0.915966
2637680,пицц,специал,364.0,411.0,0.885645
2636908,пицц,дегустац,189.0,219.0,0.863014
2637852,пицц,фунг,568.0,676.0,0.840237


In [25]:
receipt_words_total_new[receipt_words_total_new['word_x'] == 'соррентин'].head(15)

,word_x,word_y,count_xy,count_y,perc_xy
3199359,соррентин,sorrento,77.0,254.0,0.303150
3199372,соррентин,частичн,158.0,1139.0,0.138718
3199371,соррентин,частич,27.0,439.0,0.061503
3199361,соррентин,выпеч,77.0,1514.0,0.050859
3199367,соррентин,моцарелл,346.0,9922.0,0.034872
3199365,соррентин,ит,77.0,4078.0,0.018882
3199358,соррентин,gusto,77.0,4412.0,0.017452
3199363,соррентин,замороз,77.0,5065.0,0.015202
3199366,соррентин,моцар,19.0,1720.0,0.011047
3199369,соррентин,си,280.0,25614.0,0.010932


In [29]:
receipt_words_total.to_parquet(path+'receipt_words_total.parquet')